# Setup
(No need to read)

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-t8lpbx3n
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-t8lpbx3n
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 10d2f8a026d73eada861c7d51064f7e24d8f482c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
try:
    %pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python
except:
    import os; os.environ["ACCELERATE_DISABLE_RICH"] = "1"
    from IPython import get_ipython
    ipython = get_ipython()
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

  Cloning https://github.com/callummcdougall/CircuitsVis.git to /tmp/pip-req-build-xsca5hhg
  Running command git clone --filter=blob:none --quiet https://github.com/callummcdougall/CircuitsVis.git /tmp/pip-req-build-xsca5hhg
  Resolved https://github.com/callummcdougall/CircuitsVis.git to commit cc216772a66af819ff3a77038e53134f3e073af4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for circuitsvis: filename=circuitsvis-0.0.0-py3-none-any.whl size=1808565 sha256=00f6ea06383f60ca2f217e993df066f72a161cc8394eab833ee3e1580b220973
  Stored in directory: /tmp/pip-ephem-wheel-cache-jvc0p0s7/wheels/86/be/ad/78078aba9344d200aad61b63d35cdaecdec160212f039eed74
Successfully built circuitsvis
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4


In [3]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [4]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import circuitsvis as cv
import einops
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
from dataclasses import dataclass
import datasets
from IPython.display import HTML

In [5]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

Plotting helper functions:

In [6]:
import plotly.graph_objects as go

update_layout_set = {"xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth"}
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    if isinstance(tensor, list):
        tensor = torch.stack(tensor)
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    if "facet_labels" in kwargs_pre:
        facet_labels = kwargs_pre.pop("facet_labels")
    else:
        facet_labels = None
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0,labels={"x":xaxis, "y":yaxis}, **kwargs_pre).update_layout(**kwargs_post)
    if facet_labels:
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label

    fig.show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(y=utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def lines(lines_list, x=None, mode='lines', labels=None, xaxis='', yaxis='', title = '', log_y=False, hover=None, **kwargs):
    # Helper function to plot multiple lines
    if type(lines_list)==torch.Tensor:
        lines_list = [lines_list[i] for i in range(lines_list.shape[0])]
    if x is None:
        x=np.arange(len(lines_list[0]))
    fig = go.Figure(layout={'title':title})
    fig.update_xaxes(title=xaxis)
    fig.update_yaxes(title=yaxis)
    for c, line in enumerate(lines_list):
        if type(line)==torch.Tensor:
            line = utils.to_numpy(line)
        if labels is not None:
            label = labels[c]
        else:
            label = c
        fig.add_trace(go.Scatter(x=x, y=line, mode=mode, name=label, hovertext=hover, **kwargs))
    if log_y:
        fig.update_layout(yaxis_type="log")
    fig.show()

def bar(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.bar(
        y=utils.to_numpy(tensor),
        labels={"x": xaxis, "y": yaxis},
        template="simple_white",
        **kwargs).show(renderer)

In [7]:
import transformer_lens.patching as patching
from transformer_lens import evals
import math

# Load Model

In [8]:
torch.set_grad_enabled(True)

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [10]:
MAX_NUM=100
cfg = HookedTransformerConfig(
    n_layers=2,
    n_heads=1,
    d_model=128,
    d_head=128,
    attn_only=True,
    normalization_type=None,
    n_ctx=10,
    d_vocab=MAX_NUM+1, # 0,...,MAX_NUM-1, BOS
    device=device,
    seed=0
)

model = HookedTransformer(cfg)
print(model)

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-1): 2 x TransformerBlock(
      (ln1): Identity()
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_post): HookPoint()
    )
  )
  (unembed): Unembed()
)


In [11]:
# disable positional embeddings
model.pos_embed.W_pos = nn.Parameter(torch.zeros_like(model.pos_embed.W_pos))
model.pos_embed.W_pos.requires_grad = False

In [12]:
# disable biases
for name, param in model.named_parameters():
    if 'b_' in name:
        param.requires_grad = False
    print(name, param.shape, param.requires_grad)

embed.W_E torch.Size([101, 128]) True
pos_embed.W_pos torch.Size([10, 128]) False
blocks.0.attn.W_Q torch.Size([1, 128, 128]) True
blocks.0.attn.W_K torch.Size([1, 128, 128]) True
blocks.0.attn.W_V torch.Size([1, 128, 128]) True
blocks.0.attn.W_O torch.Size([1, 128, 128]) True
blocks.0.attn.b_Q torch.Size([1, 128]) False
blocks.0.attn.b_K torch.Size([1, 128]) False
blocks.0.attn.b_V torch.Size([1, 128]) False
blocks.0.attn.b_O torch.Size([128]) False
blocks.1.attn.W_Q torch.Size([1, 128, 128]) True
blocks.1.attn.W_K torch.Size([1, 128, 128]) True
blocks.1.attn.W_V torch.Size([1, 128, 128]) True
blocks.1.attn.W_O torch.Size([1, 128, 128]) True
blocks.1.attn.b_Q torch.Size([1, 128]) False
blocks.1.attn.b_K torch.Size([1, 128]) False
blocks.1.attn.b_V torch.Size([1, 128]) False
blocks.1.attn.b_O torch.Size([128]) False
unembed.W_U torch.Size([128, 101]) True
unembed.b_U torch.Size([101]) False


# Training

## Task dataset

In [13]:
BOS_TOKEN = cfg.d_vocab-1
def make_data_generator(cfg, batch_size, seed=0):
    torch.manual_seed(seed)
    while True:
        tokens = torch.randint(0, cfg.d_vocab, (batch_size, cfg.n_ctx))
        tokens[:, 0] = BOS_TOKEN
        yield tokens

print(next(make_data_generator(cfg, 4)))

tensor([[100,  61,  14,  42,  96,  32,  82,   7,  19,  94],
        [100,  54,  10,  10,  41,  90,  48,  15,  63,  98],
        [100,   8,  37, 100,  54,  93,   1,  98,  82,   1],
        [100,  74,  88, 100,  12,   9,  51,  63,  42,  25]])


## Loss fn

In [14]:
def loss_fn(logits, tokens):
    logits = logits[:, 1:, :]
    logits = logits.to(torch.float64)
    labels = tokens[:, :-1]
    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[..., None])[..., 0]
    return -correct_log_probs.mean()

with torch.no_grad():
    tokens = next(make_data_generator(cfg, 4)).to(device)
    logits = model(tokens)
    loss = loss_fn(logits, tokens)
    print(loss)

tensor(4.6229, device='cuda:0', dtype=torch.float64)


In [15]:
print("Uniform loss", np.log(cfg.d_vocab_out))

Uniform loss 4.61512051684126


## Setup optimizer / dataloader

In [16]:
lr = 1e-3
wd = 0.1
betas = (0.9, 0.95)

optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd, betas=betas)

batch_size = 256
train_data_loader = make_data_generator(cfg, batch_size)

## Training Loop

In [17]:
num_epochs = 4000

train_losses = []
for epoch in tqdm.tqdm(range(num_epochs)):
    tokens = next(train_data_loader).to(device)
    logits = model(tokens)
    loss = loss_fn(logits, tokens)
    loss.backward()
    train_losses.append(loss.item())

    optimizer.step()
    optimizer.zero_grad()

    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, train loss: {loss.item()}")

  0%|          | 0/4000 [00:00<?, ?it/s]

Epoch: 0, train loss: 4.6166477311970295
Epoch: 100, train loss: 3.0654673052579215
Epoch: 200, train loss: 0.9623682648720253
Epoch: 300, train loss: 0.3946878525033992
Epoch: 400, train loss: 0.20254613303911678
Epoch: 500, train loss: 0.11913519904140554
Epoch: 600, train loss: 0.15985523881533248
Epoch: 700, train loss: 0.06659562566443619
Epoch: 800, train loss: 0.0813063363061346
Epoch: 900, train loss: 0.054960504735594606
Epoch: 1000, train loss: 0.06978972718381639
Epoch: 1100, train loss: 0.049871853804287256
Epoch: 1200, train loss: 0.06343577050662133
Epoch: 1300, train loss: 0.07743425267511174
Epoch: 1400, train loss: 0.06036464830150533
Epoch: 1500, train loss: 0.05135188863704721
Epoch: 1600, train loss: 0.043405947526987204
Epoch: 1700, train loss: 0.04404821420206562
Epoch: 1800, train loss: 0.05700808951044047
Epoch: 1900, train loss: 0.0695774246167292
Epoch: 2000, train loss: 0.03869363237587197
Epoch: 2100, train loss: 0.045497988321708926
Epoch: 2200, train loss:

In [18]:
line(
    train_losses,
    title="Train Loss Curve",
    xaxis="Epoch", yaxis="Loss"
)

## Sanity check

In [19]:
test_sample = next(train_data_loader).to(device)
print(test_sample.shape)

torch.Size([256, 10])


In [20]:
with torch.inference_mode():
    logits = model(test_sample)
    logits = logits[:, 1:]
    labels = test_sample[:, :-1]
    preds = logits.argmax(dim=-1)
    acc = (preds == labels).float().mean()
    print("Accuracy on test sample:", acc.item())

Accuracy on test sample: 0.9921875


# Save model

In [21]:
%mkdir ../models

In [22]:
filename = "../models/rederive_position_model.pt"
torch.save(model.state_dict(), filename)

In [23]:
# check we can load in model
MAX_NUM=100
cfg = HookedTransformerConfig(
    n_layers=2,
    n_heads=1,
    d_model=128,
    d_head=128,
    attn_only=True,
    normalization_type=None,
    n_ctx=10,
    d_vocab=MAX_NUM+1, # 0,...,MAX_NUM-1, BOS
    device=device,
    seed=0
)

loaded_model = HookedTransformer(cfg)
loaded_model.load_state_dict(torch.load(filename), strict=True)

<All keys matched successfully>